In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/val/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(224,224,3)
)

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=40,
      verbose=1,
      callbacks=early_stop
      )

In [ ]:
# predicting images
for im in os.listdir("../data/test/mouth"):
    path = "../data/test/mouth/" + im
    img = image.load_img(path, target_size=(224, 224, 3))
    x = image.img_to_array(img)
    x = x/255.0
    plt.imshow(x)
    plt.show()
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images)
    print(classes)
    if classes[0]>0.5:
        print(path+ ": is yawning")
    else:
        print(path+ " is not yawning")

In [ ]:
#model.save("mobilenet_Dense__yawning_model.h5")

In [ ]:
reconstructed_model = keras.models.load_model("yawning_model.h5")

In [ ]:
test_generator = train_datagen.flow_from_directory(
        '../data/test/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

In [ ]:
reconstructed_model.evaluate(test_generator)

In [ ]:
# predicting images
for im in os.listdir("../data/test/mouth"):
    path = "../data/test/mouth/" + im
    img = image.load_img(path, target_size=(224, 224, 3))
    x = image.img_to_array(img)
    x = x/255.0
    plt.imshow(x)
    plt.show()
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = reconstructed_model.predict(images)
    print(classes)
    if classes[0]>0.5:
        print(path+ ": is yawning")
    else:
        print(path+ " is not yawning")